**Implement the Incremental Algorithm and Gauss Reduction
Algorithm. The input
should be a simplicial complex K (as a list or dictionary) or a filtra-
tion of subcomplexes $K_1 ⊂ K_2 ⊂ K_3 ⊂ · · · ⊂ K_n = K.$**

In [1]:
import numpy as np

#### **Boundary Matrix**

$$
\Delta_{i,j} = 
\begin{cases} 
1 & \text{if } \sigma_i \text{ is a face of } \sigma_j \text{ and } |\sigma_i| = |\sigma_j| - 1, \\
0 & \text{else.}
\end{cases}
$$

In [2]:
def create_boundary_matrix(simplices: list):
    set_simplices = [set(s) for s in simplices]
    matrix = np.zeros((len(simplices), len(simplices)))
    set_simplices_ordered = sorted(set_simplices, key=lambda x: len(x))
    # print(set_simplices_ordered)

    for i, sigma_1 in enumerate(set_simplices_ordered):
        for j, sigma_2 in enumerate(set_simplices_ordered):
            if sigma_1.issubset(sigma_2) and len(sigma_1)==len(sigma_2)-1:
                matrix[i, j] = 1

    return matrix, set_simplices_ordered
            

#### **The Reduced Boundary Matrix Algorithm**

For any $ j $ such that $ 1 \leq j \leq n$, define
$$
\delta(j) = \max\{i : \Delta_{i,j} \neq 0\}.
$$
If $ \Delta_{i,j} = 0 $ for all $ i $, we say that $ \delta(j) $ is undefined. 
We say that the boundary matrix $ \Delta $ is reduced if the map $ \delta $ is injective on its domain.

In [3]:
def create_reduced_boundary_matrix(simplicies: list):

    A, simplices_ordered = create_boundary_matrix(simplicies)
    # print(A)
    j = 0
    break_outer = False
    while j < A.shape[1]-1:
        rows_index_j = np.where(A[:,j] != 0)[0]
        if rows_index_j.size != 0:
            sigma_j = rows_index_j.max()
            while True:
                for i in range(j+1, A.shape[1]):
                    rows_index_i = np.where(A[:,i] != 0)[0]
                    if rows_index_i.size != 0:
                        sigma_i = rows_index_i.max()
                        if sigma_i == sigma_j:
                            A[:, i] = (A[:, i] + A[:, j]) % 2
                            j = 0
                            break_outer = True
                    if break_outer:
                        break
    
                else:
                    j += 1
                    break
                if break_outer:
                    break
        else:
            j += 1
            continue
        break_outer = False

    return A, simplices_ordered
        
        

#### **Incremental Algorithm**

In [12]:
def incremental_algorithm(simplices: list):
    A, simplices_ordered = create_reduced_boundary_matrix(simplices)
    dimension = len(simplices_ordered[-1])-1
    betti_numbers = np.zeros((dimension+1))
    # print(A)
    for j in range(A.shape[1]):
        sig = np.where(A[:,j] != 0)[0]
        d = len(simplices_ordered[j])-1
        if sig.size == 0:
            betti_numbers[d] += 1
        else:
            if d > 0:
                betti_numbers[d-1] -= 1

    return betti_numbers

#### **Test The Code**

* *For the circle*

In [6]:
circle = [
    [1], [2], [3], [1, 2], [2, 3], [3, 1]
]

In [14]:
betti_numbers = incremental_algorithm(circle)

print("Betti numbers for the circle:",betti_numbers) 

Betti numbers for the circle: [1. 1.]


 - *For the Torus*

In [22]:
torus = [
    [0], [1], [2], [3], [4], [5], [6], [7], [8],
    [0, 1], [0, 2], [0, 3], [0, 4], [0, 6], [0, 7], [1, 2], [1, 3], [1, 5], [1, 6], [1, 8], [2, 4], [2, 5], [2, 7], [2, 8], [3, 4], [3, 5], [3, 7], [3, 8], [4, 5], [4, 6], [4, 8], [5, 6], [5, 7], [6, 7], [6, 8], [7, 8],  [0, 1, 3], [0, 1, 6], [0, 2, 4], [0, 2, 7], [0, 3, 7], [0, 4, 6], [1, 2, 5], [1, 2, 8], [1, 3, 5], [1, 6, 8], [2, 4, 8], [2, 5, 7], [3, 4, 5], [3, 4, 8], [3, 7, 8], [4, 5, 6], [5, 6, 7], [6, 7, 8]
]

In [23]:
betti_numbers = incremental_algorithm(torus)

print("Betti numbers for the torus:",betti_numbers) 

Betti numbers for the torus: [1. 2. 1.]


 - *Cylinder*

In [26]:
cilinder = [
    [0], [1], [2], [3], [4], [5],
    [0, 1], [0, 2], [0, 3], [0, 5], [1, 2], [1, 4], [1, 5], [2, 3], [2, 4], [3, 4], [3, 5], [4, 5],
     [0, 1, 2], [0, 1, 5], [0, 3, 5], [1, 2, 4], [2, 3, 4], [3, 4, 5]
]

In [27]:
betti_numbers = incremental_algorithm(cilinder)

print("Betti numbers for the cilinder:",betti_numbers) 

Betti numbers for the cilinder: [1. 1. 0.]
